In [2]:
from cmdstanpy import CmdStanModel


import arviz as az
import numpy as np
import scipy.stats as stats

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Excercise 1 - generate data

In [3]:
%%writefile code_1.stan

data {
    int<lower=1> N; //number of samples
}

generated quantities {

    real theta = 0.01;
    array [N] real y;
    for (i in 1:N) {
        /* code */
        if (bernoulli_rng(theta)) {
            y[i] = normal_rng(10,10);
        }
        else {
            y[i] = normal_rng(0,1);
        }
    }
}

Writing code_1.stan


In [ ]:
%%writefile code_2.stan

data {
    int<lower=1> N; //number of samples
}

generated quantities {
    matrix[N, 3] X;

    {vector [N] zeros = rep_vector(0,N);
    for (j in 1:3) {
        X[:,j]=to_vector(normal_rng(zeros,1));
        /* code */
    }
    }

    vector [3] beta;
    beta = [2, 1, 0.5]';
    
    array [N] real y;
    y = normal_rng (X*beta, 1);
}